# Bogazici DataCamp 2020 , PyBullet - RL Workshop
## Soft Actor Critic algoritması with PyBullet Miniatur ortamında
##### Kod yapısı resmi TF Agent egitimlerinden alinmistir.  

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/7_SAC_minitaur_tutorial">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/7_SAC_minitaur_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/7_SAC_minitaur_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

<img src="https://i.ibb.co/S6dw5jd/Screenshot-2020-11-08-at-14-33-30.png" />

## Baslangic

Bugun ki workshop'ta TF Agents kütüphanesi ve Tensorflow 2.0 kullanarak pyBullet ortamında bulunan bir Reinforcement Learning Agent(Pekiştirmeli Öğrenme Çalışanı/Birimini) eğiteceğiz. 

Eğitim süresince 
  - Kurulum ve eğitim sürecini takip etmek
  - Eğitim sonrası performansı değerlendirmek
  - Colab/Jupiter notebook ortamları üzerinde video şeklinde görüntülemek
  - TF Agents yapılarında basitçe RL Agent oluşturabilmek 

Konularına değineceğiz , eğitim bitiminde bu ortam/notebook link şeklinde size sunulacaktır. Eğitim boyunca not almanız / soru sormanız daha yararlı olabilir birebir kod takibi yerine.

Not : Bu kodları  Colab ortamı yerine kişisel bilgisayarınızda da çalıştırabilirsiniz.


Yüklemeniz gereken paketler :

In [ ]:
!sudo apt-get install -y xvfb ffmpeg
!pip install gym
!pip install 'imageio==2.4.0'e
!pip install matplotlib
!pip install PILLOW
!pip install tf-agents[reverb]
!pip install pybullet

## Kurulum

Bize gerekli olan paketleri import ediyoruz.

In [ ]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf

# Alttaki kisimlar TF Agents kutuphanesinde kullanilacak bolumler.

from tf_agents.agents.ddpg import critic_network
# bu algoritmayı kullanmiyoru fakat icindeki critic ag yapisini kullanacagiz 
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.experimental.train import actor
from tf_agents.experimental.train import learner
from tf_agents.experimental.train import triggers

# gerekli yardimci paketler / fonksiyonlar
from tf_agents.experimental.train.utils import spec_utils
from tf_agents.experimental.train.utils import strategy_utils
from tf_agents.experimental.train.utils import train_utils
from tf_agents.metrics import py_metrics

# neural network ile ilgili kisimlar
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy

# datalari kullanmak icin - performansli bir veri yapisi
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils

tempdir = tempfile.gettempdir()

## Hyperparametreler

In [ ]:
env_name = "MinitaurBulletEnv-v0" # @param {type:"string"}

# Use "num_iterations = 1e6" for better results (2 hrs)
# 1e5 is just so this doesn't take too long (1 hr)
num_iterations = 100000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
collect_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 10000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5000 # @param {type:"integer"}

num_eval_episodes = 20 # @param {type:"integer"}
eval_interval = 10000 # @param {type:"integer"}

policy_save_interval = 5000 # @param {type:"integer"}

## Environment ( Ortam ) 

Pekistirmeli ogrenmede ortam dedigimiz kavram herhangi bir optimizasyon problemini ifade ediyor. Bu bir gorevin parcasi veya problemin tamamı olabilir. Ornegin robotun parmaklarini kullanarak ( manipule ederek ) dustugu yerden kalkip ziplamasi. Burada birden cok gorev parcasi var ama problem bir adet.

TF-Agents paketinin icinde standart bulunan belli basli problemler bulunuyor. Bu problemler RL algoritmalarının ortak zeminde test edilebilmesi icin onem tasiyor.  

`suites` adını verdigimiz bu ortam yukleyicilerinin icinde en cok bilinen OpenAI Gym , Atari oyunlari , Deep Mind'in kontrol paketi gibi farklı ortamlar mevcut geliyor. Tek bir string degistirerek farkli ortamı yuklemenizi sagliyor.

Bugun ki eğitim icin PyBullet fizik / robotik simulasyonunda bulunan minyatur hayvanı temalı robot ortamını yukleyecegiz.


In [ ]:
env = suite_pybullet.load(env_name)
env.reset()
PIL.Image.fromarray(env.render())

Bu ortamda amac egitecegimiz politika ile minyatur robotun dusmeden ileriye gitmesini saglayabilmek.

Herhangi bir deneme(ortam resetlenmeden ileriye gitme sureci) 1000 adım surecek sekilde ayarlandı. Robot 1000 adım durmaksızın gidebilirse ortamı cozmus sayilacaktir. Bu surecte topladigi odullerin toplami kazandigi toplam skor olacaktir.

Tabiki dusmesi veya ortam disina cikmasi gibi bir durumda 1000 adım gecmeden ortam resetlenecek sekilde ayarlandi.

Simdi egitimde kullanacagimiz ortamın detaylarına goz atalim.
 `observation` adını verdigimiz bilgiler egitim sırasında gozumuz/kulagimiz olacak ve optimizasyonu bu bilgiler isiginda gerceklestirecegiz.
 Verilen bilgilere gore politikamız/policy bize ortamda hareket etmemiz gereken / degistirmemiz gereken  `actions` uretecektir.

 

In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Goruldugu uzere ortamdan gelen "**bilgiler**" yeterli seviyede karmasik ve cozumu bulabilmek icin kısa bir egitim yeterli olmayacak. 28 sayısal deger iceriyor bunlar , 
 - Motorun bacak motorlarinin acisal degerleri
 - Ayak eklemlerinin motor degerleri
 - Hız degerleri ( acisal ve cizgisel hız) - IMU sensoru
 - Robotun belirli bolgelerinin tork degerleri.

Ful lliste icin https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/bullet/minitaur.py

Bu degerler icin normalizasyon yapılarak degerler `[-1, 1]` bandına sıkıstiriliyor.

Aynı sekilde robotun aksiyonları da `[-1, 1]` araliginda olup 8 adet istenen motor aci degerleridir.

Anlasildigi uzere bu sisteme yurutmeyi ogretmek klasik robotik algoritmaları agir muhendislik ve cok fazla guncelleme / test gerektiren bir surec. 
RL kullanarak aynı algoritma tabanları ile farklı problemleri cozebiliyoruz ve cozulen bu problemler genelde klasik algoritmalardan daha optimal sonuclar veriyor.



In [ ]:
collect_env = suite_pybullet.load(env_name)
eval_env = suite_pybullet.load(env_name)

Genelde RL sistemlerinde egitim asamasinda iki ortam birlikte yaratilir. Biri veri toplamak icin yapılırken digerinde egitim sonucu ogrenilen network ile degerlendirme / performans saptanır. Bunun sebebi ***bilgi gucu olarak yeterli olmamız / tekrar kurulum ve ayarları yapmamak / veri toplamayı durdurmamak*** gibi sıralanabilir. Bugun ki ortamlar python'da yazılmıs ve numpyarraylerini kullanan kod parcalari olacak. Bu sekilde TF agents'te bulunan Actor Learner API'lerini direk kullanabiliyoruz.

## Egitim Asamalari Ozeti
Egitim Asamalarimiz:
* Egitim datasini batchler halinde almak
* Tek GPU'da / CPU'da yapiyorsaniz burayi gecin
* Coklu GPU'da yapiyorsanız bunları sistemlere dagitmak
* Forward Step ( Bilgileri neural networke sokup cikti hesaplatmak)
* Algoritmada ki yapıya gore ortalama(mean) loss hesaplamak
* Backward Step ( Ters turev islemleri ile gradientleri hesaplayip yeni agirliklari bulmak)

İstedigimiz performansı (skoru) elde edene dek bu sekilde devam ediyor olacagiz.



### GPU'da egitim
Egitimi GPU'da yapmak istiyorsanız aktif hale getirmeniz gerekiyor.

* Edit→Notebook Settings
* GPU - > Hardware Accelerator 

In [ ]:
use_gpu = True #@param {type:"boolean"}

strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

Kullanacaginiz ayara gore ellemeyebilirsiniz.

## RL Agent - RL Calisani

Bugun egitecegimiz Soft Actor Critic algoritmasını kullanan RL calisani / agent 'ini derin ogrenme aglariyla egitmemiz gerekecek. Problemin karmasikligindan oturu genel gecer bir fonksiyon tahmincisi(approximator) yeterli olmayacaktir ( ornek agac yapilari vs ). 

Bu algoritma aynı anda egitilen 2 adet derin ogrenme agiyla egitilmektedir. Bunlardan biri **"Actor"**  yapısında kullanılacak digeri ise **"Critic"** adını verdigimiz yapida olacaktır. 

Critic adını verdigimiz yapı bize o an bulundugumuz durumda almayı dusundugumuz aksiyonun degerini verecektir `Q(s,a)`.  Bu fonksiyon derin ogrenme agi tarafindan hesaplanmaktadir ( eger problem basit bir sekilde cozulecek sekilde ise herhangi bir 2D matrix yapısında dahi tutabilirdiniz) . Fakat `28 bilgi X  8 aksiyon X (-1,1) sonsuz araligi` icin bu yapi mumkun degildir. 

Bu sebeple derin ogrenme agina bilgileri girdi/input olarak verip cikti olarak 8 aksiyonu bekleyecegiz. 

In [ ]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))
# Burdaki neural networklerin yapisi tf agents kutuphanesinde bulunmaktadir
# strag scope  datayı nasil dagittiniza ve gpu/cpu sistemlerinize gore ayarlanan
# bir parametre , suanlik dikkate almaniza gerek yok.
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None, 
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

Burda egitecegimiz `actor` yapisi ise `criticten` aldigi degerlendirmeyi alarak ve bilgileri kullanarak aksiyon arrayleri uretecektir/tahminleri.

Actor'un ag yapisi geregi urettigi sey tan-h squashed distribusyonu olacak olup biz bu distribusyondan `sampling` islemini yapacagiz olasılık distribusyonundan veri elde edecegiz.

 [Bu konuyla ilgili daha cok bilgi ariyorsaniz](https://www.tensorflow.org/probability/api_docs/python/tfp/distributions/MultivariateNormalDiag) 


In [ ]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

Elimizdeki ag yapilarini ayaga kaldirip agent/calisan degiskenlerini olusturalim.


In [ ]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

## Replay Buffer - Veri Tekrarlamak

Veri toplama sırasında sakladigimiz yararlı verileri tekrar kullanmamız RL calisanınin ogrenmesi icin cok onemli olacaktir , cunku bu yapıyı kullanmadan her politika/neural network guncellemesi yapildiginda tekrardan veri toplamaniz gerekir. Bunun sebebi PG tabanlı algoritmaların her guncellemede karar mekanizmasını degistirmeleri( DQN tarzi algoritmalarda karar her zaman max olanı almak misal) burda ise her sefer degisiyor.

Bu verileri efektif kullanabilmek icin farklı veri yapıları var , bugun gosterecegimiz olan yeni cikmis bir yapı , REVERB ismini alıyor. DeepMind ekibinin gelistirdigi cogu algoritmada rahatlikla kullanabilen bir veri yapısı, detaylarına goz atabilirsiniz.
[Reverb](https://deepmind.com/research/open-source/Reverb)


Max size bugun icin onemli bir parametre degil , daha cok CPU server clusterlarinda kullanilan asenkron data toplama boyutunu ayarliyor
```
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0))
```




In [ ]:
table_name = 'uniform_table' # uniform yani rastgele veri verebilecek sekilde 
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

Replay buffer yapımız ortamda olusacak `bilgi,odul,sonraki_bilgi,tamamlanma bilgisi` tarzinda yapilari tensorler olarak saklayacaktir.
`tf_agent.collect_data_spec`. -> Detaylı bilgi

SAC Algoritmasında(loss hesaplanmasinda) hem suanki bilgi hem sonraki bilgi gerektigi icin veri toplarkende ikili toplamamiz gerekiyor `sequence_length=2`.

In [ ]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

Simdi dataset uretimi yapacagiz. Reverb yapisi kullanip tensorflowun el verdigi fonksiyonlar ile hizlica veri toplama islemi gerceklesecek.

In [ ]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

## Politika - Karar Mekanizmalarımız

TF-Agents kutuphanesinde, politikalar RL'de kullanılan politika tanımıyla uyusmaktadir.: Verilmis bir zaman araliginda  `time_step` belirli bir aksiyon veya aksiyon distribüsyonu olusturmaktadir. 

Kullanılan ana method  `policy_step = policy.step(time_step)`  `policy_step`  -> Zaman Araligi

-   `agent.policy` —  Gercek sistemde veya test yaparken kullanacaginiz mekanizma
-   `agent.collect_policy` — Sadece data toplamak icin kullanılacak mekanizma

In [ ]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [ ]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

Politikalar/Karar mekanizmaları her ne kadar RL Calisani/ Agent icinde bulunsalarda onlardan bagimsizda olusturabilirsiniz.

Ornegin rastgele hareket secen bir karar mekanizması olusturalim. `tf_agents.policies.random_py_policy`  , her zaman araliginda rastgele bir hareket sececektir.

In [ ]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

## Actor Yapısı

Actor yapısı politikayla ortam arasindaki iliskileri kontrol eden yapıdır

  * Actor icinde bir adet ortamı temsil eden degisken bulundurur (`py_environment`) ve bir adet karar mekanizmasının kopyasını 
  * Her actor yapısı verilen seri seklindeki veri/dataset icin elinde bulunan karar mekanizmasını kullanır
  * Karar mekanizmasına yapilan guncellemeler , egitim kodu icinde `actor.run()` cagirmadan once yapılıyor
  * Ortamdan gelen yeni bilgiler veri toplama mekanizmasıyla kaydediliyor ve islem devam ediyor surekli.


Actorler calistigi surece (bilgi,aksiyon,odul) mekanizmalarını kaydediyorlar ve reverb sistemi bu numpy arraylerini sonra kullanılmak uzere sakliyor.

Bu algoritma icin ikili sekil kaydediyoruz(yukarda bahsedildi)

In [ ]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

Elimizde suan zeki bir mekanizma olmadigi icin rastgele hareket alan bir yapiyla veri toplamasi yapalım.

In [ ]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

Bir tanede birazdan akıllanmaya basladikca daha iyi veriler toplayabilecek olan collect_policy , toplama mekanizması baslatalım. 


In [ ]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

3.Actor yapımızda algoritma/ag test edilirken kullanılsın ve sonuclari  `actor.eval_metrics(num_eval_episodes)` araciligiyla log etsin.

In [ ]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

## Egitim sureci - Learners
Learner adı verilen yapı derin ogrenme agi egitimini gerceklestiren yapı. Burada gradient hesaplaması gibi asamalar gerceklesiyor. Kendi basina abstract / capsule edilmis bir yapı detayına girmeden rahatca kullanma imkanı sagliyor. Sakladigimiz veri ve elimizdeki politikayı kullanarak RL calisanının derin ogrenme agi degerlerini guncelliyor. 

In [ ]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]

agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)

## Test - Performans Analizi

3.Actorumuz olan test actoru yani ogrenilen karar mekanizmasını test edecek ve performansı ile ilgili bilgi verecek olan yapı  `actor.eval_metrics` fonksiyonu ile bize sonuc verecek
Bu tarz bir RL sisteminde genel olarak :

* Ortalama Toplam odul miktarı. Karar mekanizmasını bir (max 1000 lik yani episode) seklinde calistirinca toplam aldigi odulun ortalamasi genelde 10 kez olur .
* Ortalama bir bolum/episode ne kadar adım suruyor?



In [ ]:
def get_eval_metrics():
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  return results

metrics = get_eval_metrics()

In [ ]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

Bu modulde daha cok ornek bulunabilir bu tarz sonuc mekanizmaları ile ilgili.
[metrics module](https://github.com/tensorflow/agents/blob/master/tf_agents/metrics/tf_metrics.py)

## Egitimi Baslatalim !

Asagidaki dongu ortamdan data toplayacak ve aynı sırada toplanildikca egitim yapacak bir python dongusu.

Bu sırada belirli aralıklarda test yapılacak ve performans loglanacak (bize bilgi verilecek). Egitim bittikten sonra performans analizini video seklinde izleyecegiz. 

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]

for _ in range(num_iterations):
  # Training.
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)

  # Evaluating.
  step = agent_learner.train_step_numpy

  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))

rb_observer.close()
reverb_server.stop()

## Sonuc 


### Grafikler

Ortalama odul miktarini adıma bagli olarak grafige dokelim. Bu sayede robotun 1000 adımlık surecte ne kadar gittigini anlamis olacagiz.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()

### Videolar

RL sistemleri egitirken onemli bir faktorde egitim suresinde veya sonunda RL calisanini izlemek ve problemi cozup cozemedigini gozle gormektir. Her ne kadar odul alinsada bazen bu istenmeyen sekillerde elde edilebilir. 

Google Colab'da videoları izletmek icin asagidaki fonksiyon isinizi gorecektir.


In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

Asagidaki kod birkac episodelik test yaparak video kayıt yapacaktır:

In [ ]:
num_episodes = 3
video_filename = 'sac_minitaur.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_env.render())
    while not time_step.is_last():
      action_step = eval_actor.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_env.render())

embed_mp4(video_filename)